In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# 1. Setup Lokasi Data
base_dir = 'dataset/gambar'

# Cek dulu apakah folder benar-benar ada
print(f"Mengecek folder dataset di: {os.path.abspath(base_dir)}")
print(f"Isi folder: {os.listdir(base_dir)}")

# 2. Siapkan 'Generator' Data
# Kita sekalian potong data: 60% untuk Latihan (Training), 40% untuk Ujian (Validation)
train_datagen = ImageDataGenerator(
    rescale=1./255,      # Normalisasi warna (0-255 jadi 0-1)
    rotation_range=20,   # Putar-putar gambar (Augmentasi)
    horizontal_flip=True,
    shear_range=0.2,
    fill_mode='nearest',
    validation_split=0.4 # Split 40%
)

# 3. Load Data Training
print("\nMeload Data Training...")
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# 4. Load Data Validasi
print("\nMeload Data Validasi...")
validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

2025-12-13 12:01:01.942634: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Mengecek folder dataset di: /home/dika/Downloads/proyek-ai/dataset/gambar
Isi folder: ['rock', 'paper', 'scissors']

Meload Data Training...
Found 1314 images belonging to 3 classes.

Meload Data Validasi...
Found 874 images belonging to 3 classes.


In [ ]:
# --- STEP BARU: TRANSFER LEARNING ---
# Kita download otak pintar 'MobileNetV2' punya Google
# include_top=False artinya kita buang bagian kepalanya (klasifikasinya), kita cuma ambil matanya.
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(150, 150, 3),
    include_top=False,
    weights='imagenet'
)

# KUNCI RAHASIA: Bekukan (Freeze) otak Google ini biar ilmunya gak rusak
base_model.trainable = False

# Kita sambungin otak Google dengan otak kecil kita di ujungnya
model = tf.keras.models.Sequential([
    base_model,                                 # Mata Dewa (MobileNet)
    tf.keras.layers.GlobalAveragePooling2D(),   # Meringkas penglihatan dewa
    tf.keras.layers.Dense(128, activation='relu'), # Otak tambahan
    tf.keras.layers.Dropout(0.2),               # Biar gak terlalu PD (mencegah overfitting)
    tf.keras.layers.Dense(3, activation='softmax') # Output (Batu, Gunting, Kertas)
])

# Compile lagi (Wajib)
model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

model.summary()

In [ ]:
# Mulai Pelatihan (Training)
print("Mulai melatih model... (Bisa ditinggal ngopi sebentar ☕)")

history = model.fit(
    train_generator,
    steps_per_epoch=25,  # Berapa batch yang diambil per putaran
    epochs=10,           # Berapa kali mengulang pelajaran
    validation_data=validation_generator,
    validation_steps=5,  # Tes validasi sedikit saja biar cepat
    verbose=1
)

print("Selesai! Model sudah pintar sekarang.")

In [ ]:
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array
import matplotlib.pyplot as plt

# Daftar gambar yang akan diuji
daftar_gambar = ['tes2.jpg', 'tes3.jpg', 'tes4.jpg']

for path_gambar in daftar_gambar:
    print(f'\n========================')
    print(f'Menguji: {path_gambar}')
    print('========================')
    
    # Pre-processing (Ubah gambar jadi angka biar dimengerti model)
    # Ukuran WAJIB sama dengan saat training (150x150)
    img = load_img(path_gambar, target_size=(150, 150))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)  # Ubah jadi array 3D ke 4D (batch)
    x = x / 255.0  # Normalisasi (jangan lupa!)

    # Prediksi!
    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)

    # Terjemahkan Kode Angka ke Bahasa Manusia
    # Urutan alfabetis: [Paper, Rock, Scissors]
    print('\nHasil Prediksi:')

    # Tampilkan gambar biar keren
    plt.imshow(img)
    plt.title(path_gambar)
    plt.axis('off')
    plt.show()

    if classes[0][0] > 0.5:
        print("👉 Ini adalah KERTAS (Paper)")
    elif classes[0][1] > 0.5:
        print("👉 Ini adalah BATU (Rock)")
    elif classes[0][2] > 0.5:
        print("👉 Ini adalah GUNTING (Scissors)")
    else:
        print("🤔 Model bingung/tidak yakin.")

In [ ]:
print(train_generator.class_indices)

In [ ]:
# Simpan model MobileNet yang sudah pintar ini
# Kita kasih nama beda biar gak ketukar sama yang bodoh tadi
nama_file = 'model_rps_mobilenet_v2.h5'
model.save(nama_file)

print(f"✅ Model berhasil diamankan ke file: {nama_file}")